In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)
conll["test"][2]

Found cached dataset conll2003 (/Users/azatsultanov/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [3]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [6]:
import numpy as np
import torch

class ModelWrapper():
    def __init__(self, model, tokenizer):
        self.model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None)

    def _unite_entities(self, entities):
        if len(entities) <= 1:
            return entities

        united_result = []
        cur_entity = {key: entities[0][key] for key in ['entity', 'word', 'start', 'end']}
        for entity in entities[1:]:
            if entity['word'].startswith('##'):
                cur_entity['word'] += entity['word'].lstrip('#')
                cur_entity['end'] = entity['end']
            else:
                united_result.append(cur_entity)
                cur_entity = {key: entity[key] for key in ['entity', 'word', 'start', 'end']}
        united_result.append(cur_entity)
        return united_result

    def _convert_entities_to_bio(self, tokens, entities):
        bio_tags = []

        cur_entity_idx = 0
        for token in tokens:
            if token == entities[cur_entity_idx]['word']:
                bio_tags.append(entities[cur_entity_idx]['entity'])
                cur_entity_idx += 1
            else:
                bio_tags.append('O')
        return bio_tags

    def _postprocessing(self, model_output):
        entities = self._unite_entities(model_output)
        bio_tags = self._convert_entities_to_bio(tokens, entities)
        return bio_tags

    def predict(self, X):
        with torch.no_grad():
            ner_entitites = self.model(X)
            tags = [self._postprocessing(i) for i in ner_entitites]
            return tags

wrapped_model = ModelWrapper(model, tokenizer)

In [11]:
wrapped_model.predict([" ".join(tokens)]*1000)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O

In [14]:
def preprocessing_dataset(dataset):
    result = {'X': [], 'y_true': []}
    for sample in dataset:
        result['X'].append(' '.join(sample['tokens']))
        result['y_true'].append([CONLL_NER_TAGS[tag] for tag in sample['ner_tags']])
    return result

In [18]:
from sbe_vallib.validation.sampler.ner_sampler import NerSampler


train_data = preprocessing_dataset(conll['train'])
train_data['y_pred'] = wrapped_model.predict(train_data['X'])

oos_data = preprocessing_dataset(conll['test'])
oos_data['y_pred'] = wrapped_model.predict(oos_data['X'])



In [ ]:
import pickle

pickle.dump(train_data, './train_data.pkl')
pickle.dump(oos_data, './oos_data.pkl')

In [ ]:
sampler = NerSampler(train=train_data, oos=oos_data)
sampler.set_seed(42, bootstrap=True)